단어들은 겉의 불연속적은 형태와 달리 내부적으로 의미를 지닌다는 것을 배웠습니다. 이에 따라 단어들은 서로 유사성을 지닙니다. 데이터를 기반으로 모델의 정확도를 높여나가는 머신러닝을 자연어 처리에 적용시키려면, 데이터를 최대한 효율적으로 활용하여 모델을 학습시키는 것이 중요합니다.

특징 벡터는 단어 사이의 유사도를 구할때 아주 유용합니다. 워드넷의 그래프 구조에서 단어 사이의 거리 측정하고 이를 바탕으로 단어 사이의 유사도를 구하는 방법을 이야기했습니다. 그러면 벡터 사이의 유사도 또는 거리는 어떻게 구할 수 있을까요? 

### L1 거리

L1 거리란 L1 norm을 사용한 것으로 **$맨해튼 거리^{manhattan distance}$**라고도 합니다. 이 방법은 두 벡터의 각 차원별 값의 차이의 절대값을 모두 합한 값입니다.

<br></br>
$$d_{L1}(w,v) = \sum_{i = 1}^{d} |w_i-v_i| \\      w,v \in R^d$$
<br></br>

다음은 파이토치 텐서 $x_1, x_2$을 입력으로 받아 L1 거리를 반환해주는 코드입니다.

https://github.com/kh-kim/nlp_with_pytorch_examples/blob/master/chapter-05/word_similarity.ipynb

In [1]:
import torch

def get_l1_distance(x1, x2):
    return ((x1-x2).abs()).sum()

### L2 거리

<br></br>
$$d_{L2}(w,v) = \sqrt{\sum_{i=1}^{d} (w_i - v_i)^2} \\
w,v \in R^d $$
<br></br>

L2 거리란 우리가 가장 친숙한 거리 방법중 하나인 **$유클리디안^{Euclidean}$**입니다. 차원별 값 차이의 제곱의 합에 루트를 취한 형태입니다.

In [2]:
def get_l2_distance(x1, x2):
    return ((x1 - x2)**2).sum()**0.5

다음 그림은 L1 거리와 L2 거리를 쉽게 나타낸 것입니다. 

<br></br>
![](./images/5-7-1-l1l2.jpg)
<br></br>

### Infinity Norm

<br></br>
$$d_\infty(w,v) = max(|w_1 - v_1|, |w_2 - v_2|, ... , |w_d - v_d|) \\
w,v \in R^d$$
<br></br>

In [3]:
def get_infinity_distance(x1, x2):
    return ((x1 - x2).abs()).max()

다음 그림은 각 $L_1, L_2, L_\infty$ 별로 거리의 크기가 r일때 모습입니다. 색상별 선 위의 위치들은 모두 해당 거리 측정 방법에서 같은 거리를 나타냅니다. 보이는 바와 같이 L1 거리는 각 값들을 동시에 나타냅니다. 하지만 $L_2, L_\infty$로 갈수록 벡터내의 큰 값에 대해 더욱 집중해서 거리를 나타냅니다. 즉, 거리를 최소화 하도록 최적화를 수행한다면 $L_2, L_\infty$로 갈수록 전체 벡터중에서 큰 값이 작아지도록 최적화를 수행합니다.

<br></br>
![](./images/5-7-1-l1l2li.jpg)
<br></br>

### 코사인 유사도

코사인 유사도는 두 벡터 사이의 방향과 크기를 모두 고려하는 방법입니다.

<br></br>
$$sim_{cos}(w,v)=\frac{{w} {v}}{|w||v|} = \frac{w}{|w|} \frac{v}{|v|} \\
= \frac{\sum_{i=1}^{d} w_i v_i}{{\sqrt{\sum_{i=1}^{d} w_i^2}}{\sqrt{\sum_{i=1}^{d} v_i^2}}}$$
<br></br>

코사인 유사도가 1에 가까울수록 방향은 일치하고, 0에 가까울수록 직교이며, -1에 가까울수록 반대 방향임을 의미합니다. 이와 같이 코사인 유사도는 방향과 크기 모두를 고려하는 만큼 자연어 처리에서 가장 널리 쓰이는 유사도 측정 방법입니다.

하지만 수식의 연산 값이 비싼 편에 속합니다. 따라서 벡터 차원의 크기가 클수록 연산량이 부담됩니다.

희소 벡터일 경우 여기서 가장 큰 문제가 나타납니다. 윗변이 벡터 곱으로 표현됨으로 0이 들어간 차원이 많은 벡터는 직교하면서 곱의 값이 0이 되므로, 정확한 유사도 또는 거리를 반영하지 못합니다.

In [4]:
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**0.5 * (x2**2).sum()**0.5)

### 자카드 유사도

<br></br>
$$sim_{jaccard}(w,v) = \frac{|w \cap v|}{|w \cup v|} \\
= \frac{|w \cap v|}{|w| + |v| - |w \cap v|} \\
\approx \frac{\sum_{i=1}^{d} min(w_i,v_i)}{\sum_{i=1}^{d} max(w_i,v_i)}$$
<br></br>

자카드 유사도는 두 집합간의 유사도를 구하는 방법입니다. 수식의 윗변에는 두 집합의 교집합 크기가 있고, 이를 밑변에서는 두 집합의 합집합 크기로 나눕니다. 이떄 특징 벡터의 각 차원이 집합의 요소가 될 것입니다.

In [5]:
def get_jaccard_similarity(x1, x2):
    return torch.stack([x1,x2]).min(dim = 0)[0].sum() / torch.stack([x1, x2]).max(dim = 0)[0].sum()